In [47]:
import torch
import torchvision
from skimage import io

from torch.utils.data import DataLoader, Dataset

from database_functions.databasereader import DatabaseReader

In [48]:
db_reader = DatabaseReader('crepcheque')

get_images_query = """SELECT
                            r.crep_id,
                            r.raw_brand_text,
                            i.image_id,
                            i.image_type,
                            i.image_file_path
                        FROM raw_creps r
                        LEFT JOIN
                            images i
                            ON r.crep_id = i.crep_id

                        WHERE
                            r.images IS NOT NULL
                            AND r.images_processed IS NOT NULL
                            AND i.image_downloaded = true
                        """

samples_df = db_reader.send_query(query=get_images_query, return_as_df=True)

In [49]:
samples_df.head()

,crep_id,raw_brand_text,image_id,image_type,image_file_path
0,16929,Nike,172179,Additional,/home/max/hdd-data1/images/original/16929-1721...
1,16915,Jordan,172161,Additional,/home/max/hdd-data1/images/original/16915-1721...
2,16915,Jordan,172165,Additional,/home/max/hdd-data1/images/original/16915-1721...
3,16915,Jordan,172169,Additional,/home/max/hdd-data1/images/original/16915-1721...
4,16915,Jordan,172173,Additional,/home/max/hdd-data1/images/original/16915-1721...


In [50]:
class ShoeDataSet(Dataset):
    
    def __init__(self, data_df,transform = None):
        
        self.data_df = data_df
        self.transform = transform
    
    def __len__(self):
        return len(self.data_df)
    
    def __getitem__(self, idx):
        
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        print(type(idx))
        
        img_name = self.data_df.iloc[idx, 4] + ".jpg"
        print(img_name)
        image = io.imread(img_name)
#         image = 
        label = self.data_df.iloc[idx, 1]
        
        sample  ={"image":image, "label":label}
        
        if self.transform:
            sample = self.transform(sample)
        
        return sample
        

In [4]:
"""
1) transorms
2) totensor
3) normalize
4) scale

"""

In [51]:
shoe_set = ShoeDataSet(samples_df, transform=torchvision.transforms.Resize((300,300)))

In [56]:
test_loader = DataLoader(shoe_set, shuffle=True, batch_size = 10)

In [57]:
for data,label in test_loader:
    print(data)

<class 'int'>
/home/max/hdd-data1/images/original/8848-90419-original.jpg


TypeError: img should be PIL Image. Got <class 'dict'>

In [6]:
transforms = torchvision.transforms.Compose([
    torchvision.transforms.RandomPerspective(p = 0.3), # randomly change img perspective
    torchvision.transforms.,
    
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225]),
])

In [ ]:
for i in samples_df